In [34]:
from llama_index import SimpleDirectoryReader, GPTListIndex, GPTVectorStoreIndex, LLMPredictor, PromptHelper, ServiceContext, StorageContext, load_index_from_storage
import os
import gradio as gr
from langchain.embeddings import OpenAIEmbeddings
from langchain import OpenAI

In [25]:
from dotenv import load_dotenv

load_dotenv('.env')

os.environ.get('OPENAI_API_KEY')

print('Llave cargada')

Llave cargada


In [14]:
docs = SimpleDirectoryReader('datos').load_data()
docs

[Document(text='La Importancia De La Inteligencia Artificial en la Vida Del Ser\nHumano\nPara iniciar déjame hacerte una pregunta ¿cuál es tu red social favorita? Sea cual sea, todas utilizan\ninteligencia artificial. La Inteligencia artificial logra que las máquinas tengan conocimientos de tus\nhábitos, desde procesos básicos hasta situaciones más avanzadas como el reconocimiento de tu voz,\nentre otras cosas. Pero ¿qué es la Inteligencia Artificial? ¿por qué es importante para el ser humano?\nEn este artículo, se responderán estas cuestiones.\nInteligencia Artificial\nLa Inteligencia artificial es la combinación de algoritmos desarrollados con la finalidad de crear\nprogramas y mecanismos que tengan capacidades similares al del ser humano. En un concepto\npráctico, se puede decir que la IA, es que las máquinas hagan y piensen como seres humanos.\nJohn McCarthy en 1956 emitió el término Inteligencia Artificial y lo definió como “la ciencia y el ingenio\nde hacer máquinas inteligentes,

In [15]:
type(docs)

list

In [16]:
max_input = 4098
tokens = 256
chnk_size = 600
max_chnk_overlap = 20

In [17]:
prompt_helper = PromptHelper(max_input,tokens,max_chnk_overlap,chunk_size_limit=chnk_size)

In [26]:
modelo = LLMPredictor(llm=OpenAI(temperature=0,model_name='text-ada-001',max_tokens=tokens)) # type: ignore
contexto = ServiceContext.from_defaults(llm_predictor=modelo,prompt_helper=prompt_helper)
index_model = GPTVectorStoreIndex.from_documents(docs,service_context=contexto)
index_model.storage_context.persist('modelo')

In [35]:
def chatbot(inputText):
    # rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir='modelo')
    modelo = load_index_from_storage(storage_context)
    query_engine = modelo.as_query_engine()
    respuesta = query_engine.query(inputText)
    return respuesta

In [38]:
app = gr.Interface(fn=chatbot,
            inputs=gr.Textbox(lines=5,label='Ingresa una peticion'),
            outputs='text',
            title='ChatBot')

app.launch(share=False)

Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.
